In [1]:
from cobra.io import read_sbml_model
from pathlib import Path
import os
import cobra

cobra.Configuration().solver = "gurobi"

# change working directory to context_specific_models folder
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# create a dictionary with the filenames (keys) and models (values)
folder = Path().cwd()
files = [f for f in os.listdir(folder / 'cs_mods') if '.xml'in f]
models = {f:read_sbml_model(folder / 'cs_mods' / f ) for f in files}
k = list(models.keys())

# add model names as IDs
models_names = ['Human1', 'Schwan (1)', 'Schwan (2)', 'Melanocytes', 'Endothelial','Muscle','Fibroblasts',\
                'RPE','B cells','T + NK cells', 'Monocytes + Macrophages','Mast','Retina','RPE + Choroid',\
                'PR rod','Amacrine','Bipolar','PR cone', 'Horizontal', 'Müller','Ganglion']

for i in range(len(models)):
    models[k[i]].id = models_names[i]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [2]:
import pandas as pd

mod = models[k[0]].copy()

# add back any lost exchange reactions
def add_all_EX_rxns(basis_model,model):
    EX_rxns = [r for r in basis_model.reactions if len(r.products)==0]
    EX_rxns_model = [r.id for r in model.reactions if len(r.products)==0]
    missing_rxns = [r.copy() for r in EX_rxns if r.id not in EX_rxns_model]
    model.add_reactions(missing_rxns)
    return model

# add ATP demand reactions, lost exchange reactions to all models
for k_i in k:
    models[k_i].add_reactions([mod.reactions.get_by_id('MAR03964').copy()])
    models[k_i] = add_all_EX_rxns(mod,models[k_i])
    for r in [r for r in models[k_i].reactions if len(r.products)==0]: r.bounds = (0,1000)     # close lower bound for all exchange reactions (metabolites can leave the system but not enter)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpol7r9bpv.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros


Ignoring reaction 'MAR03964' since it already exists.


In [23]:
# make RPE_PR model

# copy RPE, PR, and full Human1 models
mod = models['Human-GEM.xml'].copy()
mod_RPE = models['model_06_MeanExpression_RPE__VoigtEtAl2019_.xml'].copy()
mod_PR = models['model_13_MeanExpression_RodPhotoreceptors__LiangEtAl_2019_.xml'].copy()

def remove_compartment(model, compartment):
# remove compartment from model
    rxns_n = [r.id for r in model.reactions if compartment in r.compartments]
    mets_n = [m for m in model.metabolites if compartment in m.compartment]
    model.remove_reactions(rxns_n)
    model.remove_metabolites(mets_n)
    return model

# remove nucleus from PR model
mod_PR = remove_compartment(mod_PR, 'n')

def add_id_suffix(model, suffix):
    for r in model.reactions:
        r.id = r.id + suffix
    for m in model.metabolites:
        m.id = m.id + suffix
        m.compartment = m.compartment + suffix
    return model
# add '_PR' to PR model IDs and '_RPE' to RPE model IDs
mod_PR = add_id_suffix(mod_PR, '_PR')
mod_RPE = add_id_suffix(mod_RPE, '_RPE')

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpjn45sna_.lp
Reading time = 0.05 seconds
: 8369 rows, 26140 columns, 111668 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpa4ahf55u.lp
Reading time = 0.03 seconds
: 5517 rows, 14194 columns, 54382 nonzeros
Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp03refj30.lp
Reading time = 0.02 seconds
: 5013 rows, 12490 columns, 46478 nonzeros


C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [24]:
# add RPE_PR interface reactions to RPE model (rxn_RPE_PR = RPE <--> RPE_PR interface)
for rxns in  [r for r in mod_RPE.reactions if 'e_RPE' in r.reaction]:     
    rxn = rxns.copy()
    rxn.id = rxn.id.replace('_RPE','_RPE_PR')
    for p in rxn.products:
        if 'e_RPE' in p.id:
            p.id = p.id.replace('e_RPE','e_RPE_PR')
            p.compartment = 'e_RPE_PR'
    for r in rxn.reactants:
        if 'e_RPE' in r.id:
            r.id = r.id.replace('e_RPE','e_RPE_PR')
            r.compartment = 'e_RPE_PR'
    mod_RPE.add_reactions([rxn])
    
# add RPE_PR interface reactions to PR model (rxn_PR-RPE = PR <--> RPE_PR interface)
for rxn in [r for r in mod_PR.reactions if 'e_PR' in r.reaction]:
    rxn.id = rxn.id.replace('_PR','_PR_RPE')
    for p in rxn.products:
        if 'e_PR' in p.id:
            p.id = p.id.replace('e_PR','e_RPE_PR')
            p.compartment = 'e_RPE_PR'
    for r in rxn.reactants:
        if 'e_PR' in r.id:
            r.id = r.id.replace('e_PR','e_RPE_PR')
            r.compartment = 'e_RPE_PR'

In [32]:
# fuse RPE and PR model
from cobra import Model
# mod_RPE_PR = Model('RPE_PR model')
mod_RPE_PR = mod_RPE.copy()
mod_RPE_PR.id = 'RPE_PR'
mod_RPE_PR.add_reactions(mod_PR.reactions)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmpv9w47k54.lp
Reading time = 0.10 seconds
: 7182 rows, 19688 columns, 65842 nonzeros


In [33]:
# list reaction IDs to be deleted (because their reactions are duplicated)
import pandas as pd
df = pd.DataFrame([[r.id, r.reaction] for r in mod_RPE_PR.reactions \
                   if r.compartments == {'e_RPE_PR'}], columns=['id','reaction'])
l  = list(df['id'][df['reaction'].duplicated()])

# delete duplicated reactions and change reaction IDs (suffix: _eRPE_PR) 
# of the reactions that only involve the e_RPE_PR compartment (interface)
for r in [r for r in mod_RPE_PR.reactions if r.compartments=={'e_RPE_PR'}]: 
    if r.id in l:
        mod_RPE_PR.remove_reactions(r.id)
    elif '_RPE_PR' in r.id:
        r.id = r.id.replace('_RPE_PR','_eRPE_PR')
    elif '_PR-RPE' in r.id:
        r.id = r.id.replace('_PR_RPE','_eRPE_PR')
        ,
def fix_compartment_dict(model):
    new_compartment_dict = model.compartments.copy()
    for k in list(new_compartment_dict.keys()):
        new_compartment_dict[k] = k 
    model.compartments = new_compartment_dict
    return model
fix_compartment_dict(mod_RPE_PR)

# close lower bounds of exchange reactions into RPE_PR interface space (nothing allowed in from outside)
rxns = [r for r in mod_RPE_PR.reactions if len(r.products)==0 if r.lower_bound<0 if 'eRPE_PR' in r.id]
for rxn in rxns: rxn.lower_bound = 0

# set objective function
mod_RPE_PR.objective = 'MAR03964_PR'

C:\Users\prins\miniconda3\envs\RPE-PR\lib\site-packages\cobra\core\model.py:778: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [34]:
# set objective function
mod_RPE_PR.objective = 'MAR03964_PR'

In [35]:
mod_RPE_PR 

Name,RPE_PR
Memory address,198de490670
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [36]:
from cobra.io import write_sbml_model, read_sbml_model
write_sbml_model(mod_RPE_PR, "mod_RPE_PR.xml")
read_sbml_model('C:/Users/prins/GitHub/Human1_RPE-PR/mod_RPE_PR.xml')

Name,RPE_PR
Memory address,19b60590640
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [30]:
cobra.io.sbml.validate_sbml_model('C:/Users/prins/GitHub/Human1_RPE-PR/mod_RPE_PR.xml')

(<Model RPE at 0x19a91369840>,
 {'SBML_FATAL': [],
  'SBML_ERROR': [],
  'SBML_SCHEMA_ERROR': [],
  'SBML_WARNING': [],
  'COBRA_FATAL': [],
  'COBRA_ERROR': [],
  'COBRA_WARNING': [],
  'COBRA_CHECK': []})

In [ ]:

# check compartment info M_MAM02040e_RPE__45__PR

mod_RPE_PR.metabolites.get_by_id('MAM02040e_RPE_PR').compartment



In [ ]:
# set bounds Exchange reactions (lower bounds -1000, upper bounds based on file info)  

# import excel file with info on exchange upper bounds 
# file contains 'id' col with IDs and an 'upper_bound' col with upper bounds to modify
bounds = pd.read_excel(folder / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

for i in range(len(bounds)): 
    models[k_i].reactions.get_by_id(bounds['id'][i]).bounds = (-bounds['upper_bound'][i],1000)   

In [ ]:
# open RPE exchange reactions
for i in range(len(bounds)): 
    mod_RPE_PR.reactions.get_by_id(bounds['id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)   

In [ ]:
# ATP demand reactions RPE
mod_RPE_PR.reactions.get_by_id('MAR03964_RPE')

In [ ]:
# ATP demand reactions PR
mod_RPE_PR.reactions.get_by_id('MAR03964_PR')

In [ ]:
# glucose exchange reaction RPE
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

In [ ]:
# OBJECTIVE 
# set maximizing the ATP demand in PR as objective
mod_RPE_PR.objective = mod_RPE_PR.reactions.get_by_id('MAR03964_PR').flux_expression
solution = mod_RPE_PR.optimize()
print(solution)

In [ ]:
def get_vmh_id(reaction):
    vmh_id = ''
    if 'vmhreaction' in reaction.annotation.keys(): 
        vmh_id = reaction.annotation['vmhreaction']
    return  vmh_id
    
df= pd.DataFrame([[get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod_RPE_PR.reactions],\
                 index = [r.id  for r in mod_RPE_PR.reactions],
             columns=['vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
objectives = {mod_RPE_PR.reactions.get_by_id('MAR03964_PR'): 2, mod_RPE_PR.reactions.get_by_id('MAR03964_RPE'): 1}

from cobra.flux_analysis import flux_variability_analysis
mod_RPE_PR.objective = 'MAR03964_PR'
mod_RPE_PR.objective.expression
fva = flux_variability_analysis(mod_RPE_PR, loopless=True)

In [ ]:
pd.merge(df, solution.to_frame(), left_index=True, right_index=True)

In [ ]:
solution.to_frame()

In [ ]:
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
solution_loopless = loopless_solution(mod_RPE_PR)

import pandas
df = pandas.DataFrame(dict(loopless=solution_loopless.fluxes, nominal=solution.fluxes))

In [ ]:
solution_loopless

In [ ]:
df

In [ ]:
# 
#with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    solution_add_loopless = mod_RPE_PR.optimize()
#print("loopless solution: status = " + solution_add_loopless.status)
#print("loopless solution flux: v3 = %.1f" % solution_add_loopless.fluxes["v3"])
# solution_add_loopless.fluxes

In [ ]:
mod_RPE_PR.summary()

In [ ]:
solution_loopless

In [ ]:
from cobra.flux_analysis import pfba
solution_pfba = pfba(mod_RPE_PR)
print("parsimonious solution: status = " + solution_pfba.status)

In [ ]:
solution_pfba

In [ ]:
solution_pfba.fluxes

In [ ]:
solution_pfba.to_frame()

In [ ]:
s = mod_RPE_PR.summary(fva=0.95)

In [ ]:
solution.to_frame()

In [ ]:
s.to_frame()

In [ ]:
mod_RPE_PR.reactions.get_by_id('MAR09034_RPE')

In [ ]:
# with mod_RPE_PR:
#    add_loopless(mod_RPE_PR)
#    try: solution = mod_RPE_PR.optimize()
#    except: print('model is infeasible')

In [ ]:
# FVA
#from cobra.flux_analysis import flux_variability_analysis
#flux_variability_analysis(mod_RPE_PR,loopless=True)

In [ ]:
# list all exchange and demand reactions matching Human1 and VMH IDs
def get_vmh_id(reaction):
    if 'vmhreaction' in reaction.annotation.keys(): vmh_id = reaction.annotation['vmhreaction']
    else: vmh_id = ''
    return vmh_id
df_EX_rxns = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products) == 0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns

In [ ]:
# ATP metabolites
ATP_mets = [[met.id,met.name] for met in mod_RPE_PR.metabolites if met.name=='ATP']
pd.DataFrame(ATP_mets,columns=['ID','name'])

# ATP reactions
ATP_rxns = [[r.id,get_vmh_id(r),r.name,r.subsystem,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
            for r in mod_RPE_PR.reactions if 'MAM01371' in r.reaction]

pd.DataFrame(ATP_rxns,columns=['id','vmh id', 'name', 'subsystem', 'reaction (id)', 'reaction (name)'])

In [ ]:
# list all exchange and demand reactions
df_EX_rxns_human1 = pd.DataFrame([[r.id,get_vmh_id(r),r.name,r.reaction,r.build_reaction_string(use_metabolite_names = True)] \
              for r in mod.reactions if len(r.products)==0],columns=['id','vmh id','name','reaction','reaction (names)'])
df_EX_rxns.to_clipboard(excel=True, sep=None)
df_EX_rxns

In [ ]:
# match VMH_IDs from bounds file to Human1 IDs to create 'R3D301_EX_rxns_RPE_PR.xlsx'

# import excel file with info on bounds
bounds = pd.read_excel(folder / 'rxn_bounds' / 'files PL' / 'R3D301_EX_rxns.xlsx')

# vmh_id lookup table + look up RPE rxn
df_vmh_id = pd.DataFrame([[get_vmh_id(r),r.id,r.id+'_RPE'] for r in mod.reactions if 'EX_' in get_vmh_id(r)],columns=['vmh_id','human1_id','id'])
merged_df = df_vmh_id.merge(bounds, how = 'right', on = ['vmh_id'])
merged_df.to_clipboard(excel=True, sep=None)
merged_df

In [ ]:
# biomass reaction
mod.reactions.get_by_id('MAR04413') 

pd.DataFrame([[r.id,r.name,r.reaction,r.reaction,r.build_reaction_string(use_metabolite_names = True)]\
              for r in mod_RPE_PR.reactions if 'biomass' in r.name])

In [ ]:
mod.reactions.get_by_id('MAR13082') 